# Basic data

In [57]:
import pandas as pd
import os
from sqlalchemy import create_engine

connection = os.environ['hydra_connection'].replace('postgresql','postgresql+psycopg')

engine = create_engine(connection)

with engine.begin() as conn:
    try:
        # Query the database and obtain data as Python objects.
        data = pd.read_sql_query('SELECT * FROM fact_hospital_events ORDER BY admission_test_date ASC, discharge_date ASC, total_cost DESC;', conn).style.set_caption("fact_hospital_events")
        data_alternative = pd.read_sql_query('SELECT * FROM fact_hospital_events_alternative ORDER BY total_stay_cost DESC, total_tests_cost DESC;', conn).style.set_caption("fact_hospital_events_alternative")

        display(data)
        display(data_alternative)

    
    except BaseException as err:
        conn.rollback()
        raise err
    
    else:
        conn.commit()
    
    finally:
        conn.close()

,event_id,patient_id,test_id,doctor_npi_number,admission_test_date,discharge_date,event_type,total_cost
0,10,2222,nan,None,2023-01-01 00:00:00,2023-01-01 00:00:00,stay,13.000000
1,9,1111,nan,None,2023-01-01 00:00:00,2023-01-10 00:00:00,stay,149.000000
2,16,1111,2.000000,1245319599,2023-01-01 00:00:00,NaT,test,5.000000
3,12,2222,1.000000,0265319599,2023-01-01 00:00:00,NaT,test,5.000000
4,14,1111,3.000000,1245319599,2023-01-01 00:00:00,NaT,test,3.000000
5,13,1111,2.000000,1245319599,2023-01-01 00:00:00,NaT,test,2.000000
6,11,3333,nan,None,2023-01-02 00:00:00,2023-01-06 00:00:00,stay,72.000000
7,15,3333,3.000000,1245319599,2023-01-02 00:00:00,NaT,test,3.000000


,event_id,patient_id,admission_test_date,discharge_date,total_stay_cost,total_tests_cost
0,4,1111,2023-01-01 00:00:00,2023-01-10 00:00:00,149.000000,10.000000
1,6,3333,2023-01-02 00:00:00,2023-01-06 00:00:00,72.000000,3.000000
2,5,2222,2023-01-01 00:00:00,2023-01-01 00:00:00,13.000000,5.000000


## Stays + tests costs data from fact_hospital_events

In [58]:
with engine.begin() as conn:
    try:
        # Query the database and obtain data as Python objects.
        data = pd.read_sql_query('SELECT s.patient_id, s.admission_test_date, s.discharge_date, s.total_cost AS total_stay_cost,\
                                SUM(t.total_cost) AS total_tests_cost\
                                FROM fact_hospital_events s\
                                LEFT JOIN fact_hospital_events t ON t.patient_id = s.patient_id AND t.event_type = \'test\' AND t.admission_test_date BETWEEN s.admission_test_date AND s.discharge_date\
                                WHERE s.event_type = \'stay\'\
                                GROUP BY s.patient_id, s.admission_test_date, s.discharge_date, s.total_cost\
                                ORDER BY total_stay_cost DESC, total_tests_cost DESC;', conn)
        display(data)

    
    except BaseException as err:
        conn.rollback()
        raise err
    
    else:
        conn.commit()
    
    finally:
        conn.close()

,patient_id,admission_test_date,discharge_date,total_stay_cost,total_tests_cost
0,1111,2023-01-01,2023-01-10,149.0,10.0
1,3333,2023-01-02,2023-01-06,72.0,3.0
2,2222,2023-01-01,2023-01-01,13.0,5.0


## doctor/practice data

In [52]:
with engine.begin() as conn:
    try:
        # Query the database and obtain data as Python objects.
        data = pd.read_sql_query('SELECT s.doctor_npi_number, d.name, d.practice, SUM(total_cost) AS total_cost\
                                FROM fact_hospital_events s\
                                INNER JOIN dim_doctor d ON d.doctor_npi_number = s.doctor_npi_number\
                                WHERE s.event_type = \'test\'\
                                GROUP BY s.doctor_npi_number, d.name, d.practice\
                                ORDER BY total_cost DESC;', conn)
        display(data)

    
    except BaseException as err:
        conn.rollback()
        raise err
    
    else:
        conn.commit()
    
    finally:
        conn.close()

,doctor_npi_number,name,practice,total_cost
0,1245319599,Julia Styles,radiology,13.0
1,0265319599,Greg Brown,biochemistry,5.0


## Patient's stays consolidated cost

In [59]:
with engine.begin() as conn:
    try:
        # Query the database and obtain data as Python objects.
        data = pd.read_sql_query('SELECT s.patient_id, s.admission_test_date, s.discharge_date,\
                                (SUM(t.total_cost) + s.total_cost) AS total_cost\
                                FROM fact_hospital_events s\
                                LEFT JOIN fact_hospital_events t ON t.patient_id = s.patient_id AND t.event_type = \'test\' AND t.admission_test_date BETWEEN s.admission_test_date AND s.discharge_date\
                                WHERE s.event_type = \'stay\'\
                                GROUP BY s.patient_id, s.admission_test_date, s.discharge_date, s.total_cost\
                                ORDER BY total_cost DESC;', conn)
        display(data)

    
    except BaseException as err:
        conn.rollback()
        raise err
    
    else:
        conn.commit()
    
    finally:
        conn.close()

,patient_id,admission_test_date,discharge_date,total_cost
0,1111,2023-01-01,2023-01-10,159.0
1,3333,2023-01-02,2023-01-06,75.0
2,2222,2023-01-01,2023-01-01,18.0
